In [1]:
import numpy as np
import pandas as pd
import re
from keras.preprocessing import sequence
import pickle

Using TensorFlow backend.


In [0]:
#loaing Data
train_df = pd.read_csv('sample_data.csv')

In [4]:
train_df[:5]

,Q_id,Question,Answer,Target,Respose_number
0,10000,symptoms of a dying mouse,This can be fatal quite quickly to mice. 1 It...,0.0,0.0
1,10000,symptoms of a dying mouse,The symptoms of mites include: 1 excessive sc...,0.0,1.0
2,10000,symptoms of a dying mouse,Symptoms of Dog and Cat Poisoning. The symptom...,0.0,2.0
3,10000,symptoms of a dying mouse,The symptoms of mites include: excessive scrat...,0.0,3.0
4,10000,symptoms of a dying mouse,Seizures and neurologic symptoms are caused by...,0.0,4.0


## Function to make predictions made on Raw Data

In [0]:
def final_fun_1(X):
    '''Function takes raw data as input and do all preprocessing and returns models prediction'''
    data = X.copy()
    preprocessed_Questions = []
    for sentance in (data['Question'].values):
        sent = re.sub('[^A-Za-z0-9]+', ' ', sentance)
        sent = ' '.join(e.lower() for e in sent.split())
        preprocessed_Questions.append(sent)
    
    preprocessed_Answers = []
    for sentance in (data['Answer'].values):
        sent = re.sub('[^A-Za-z0-9]+', ' ', sentance)
        sent = ' '.join(e.lower() for e in sent.split())
        preprocessed_Answers.append(sent)
        
    data['Question'] = preprocessed_Questions
    data['Answer'] = preprocessed_Answers
    
    
    Question_keywords = np.load(r'Questin_Answer_10000_keywords/Question_keywords.npy',allow_pickle = True)
    Answer_keywords = np.load(r'Questin_Answer_10000_keywords/Answer_keywords.npy',allow_pickle = True)
    
    que_which_has_ly_keywords = []
    for sentance in data['Question']:
        sent = ' '.join(e for e in sentance.split() if e in Question_keywords)
        que_which_has_ly_keywords.append(sent)
    
     
    ans_which_has_ly_keywords = []
    for sentance in data['Answer']:
        sent = ' '.join(e for e in sentance.split() if e in Answer_keywords)
        ans_which_has_ly_keywords.append(sent)
        
    data['Question'] = que_which_has_ly_keywords
    data['Answer'] = ans_which_has_ly_keywords
        
        
    #Setting numerical value to each word in questions
    ecoded_ques = []
    for train_ques in data['Question']:
        ecoded_ques.append(list(map(lambda x: list(Question_keywords).index(x)+1,train_ques.split(" "))))
    
    ecoded_ans = []
    for train_ans in data['Answer']:
        ecoded_ans.append(list(map(lambda x: list(Answer_keywords).index(x)+1,train_ans.split(" "))))
    
    data['Question_vector'] = ecoded_ques
    data['Answer_vector'] = ecoded_ans
    
    
    #truncate and/or pad input sequences
    Max_Question_length = 24
    ques_vect_1 = sequence.pad_sequences(list(data.Question_vector), maxlen=Max_Question_length)
    ques_vect = [np.hstack((x,10001)) for x in ques_vect_1]
        
    # truncate and/or pad input sequences
    Max_Answer_length = 163
    ans_vect = sequence.pad_sequences(data.Answer_vector, maxlen=Max_Answer_length)
    
    
    #load config & weight of best model
    with open('Training_Embedding_Layer/Best_model/config.p', 'rb') as fp:
        config = pickle.load(fp)

    with open('Training_Embedding_Layer/Best_model/Best_weights_initial.p', 'rb') as fp:
        weights = pickle.load(fp)

    #loading weights to model
    import tensorflow as tf
    model = tf.keras.Model.from_config(config)
    model.set_weights(weights)
    
    ques_vect_ds = list(tf.data.Dataset.from_tensor_slices((ques_vect)).batch(1024).as_numpy_iterator())
    ans_vect_ds = list(tf.data.Dataset.from_tensor_slices((ans_vect)).batch(1024).as_numpy_iterator())
    

    @tf.function
    def predict(input_vec):
        '''Taks in a batch of vector ,pass it into model and returns the predicted value'''
        predictions = model(input_vec, training=False)
        return predictions

    #stack the output of each batch
    def predicted_y(ques_vect,ans_vect):
        '''Takes in the a two vector(Question,answe vecror) and returns the predicted result'''
        ls = []
        for batch in range(len(ques_vect)):
            test_ques_vec = ques_vect[batch]
            test_ans_vec = ans_vect[batch]
            ls.append(predict([test_ques_vec,test_ans_vec]))

        pred_y = ls[0]
        for i in range(1,len(ls)):
            pred_y = np.vstack((pred_y,ls[i]))
        return pred_y
    
    predicted_y = predicted_y(ques_vect_ds,ans_vect_ds)
    
    for_output = np.zeros_like(predicted_y,dtype = np.int32)
    for i in range(0,predicted_y.shape[0],10):
        maxi = np.argmax(predicted_y[i:i+10])
        for_output[i+maxi] = 1


    return np.array(for_output[:,0]),np.array(predicted_y[:,0])

In [6]:
binary_output,probablity_output = final_fun_1(train_df.iloc[:91,:])
print("Binary Value output : \n", binary_output)
print("Probablity Value output : \n", probablity_output)

Binary Value output : 
 [0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1]
Probablity Value output : 
 [0.13777828 0.12637258 0.15753806 0.1403307  0.1734311  0.1083639
 0.10261139 0.13404188 0.17250362 0.1126875  0.04208165 0.14256439
 0.23313549 0.04767594 0.09345442 0.10812151 0.21852213 0.04989487
 0.05162087 0.07677156 0.13927269 0.1449508  0.09386876 0.101661
 0.11374235 0.07400876 0.08601692 0.07550621 0.10349679 0.18773758
 0.05309692 0.13235503 0.12124524 0.03648993 0.01339278 0.00440446
 0.00372639 0.02458245 0.00892746 0.01411194 0.09807554 0.11666846
 0.01803702 0.09841385 0.0498164  0.04485628 0.0987055  0.04379711
 0.12001166 0.05062103 0.05347881 0.05677226 0.1159744  0.07599413
 0.2465117  0.2759447  0.11140323 0.12944421 0.10867387 0.21454352
 0.04676348 0.04299769 0.04698983 0.08341014 0.03593683 0.07478622
 0.01936671 0.03458306 0.026

## Function  which returns final_metric computed on X ( Raw Data) and Y (target variable)

In [0]:
def final_fun_2(X):
    '''Functions takes raw data as input, preprocesses the dat and returns loss'''
    data = X.copy()
    preprocessed_Questions = []
    for sentance in (data['Question'].values):
        sent = re.sub('[^A-Za-z0-9]+', ' ', sentance)
        sent = ' '.join(e.lower() for e in sent.split())
        preprocessed_Questions.append(sent)
    
    preprocessed_Answers = []
    for sentance in (data['Answer'].values):
        sent = re.sub('[^A-Za-z0-9]+', ' ', sentance)
        sent = ' '.join(e.lower() for e in sent.split())
        preprocessed_Answers.append(sent)
        
    data['Question'] = preprocessed_Questions
    data['Answer'] = preprocessed_Answers
    
    
    Question_keywords = np.load(r'Questin_Answer_10000_keywords/Question_keywords.npy',allow_pickle = True)
    Answer_keywords = np.load(r'Questin_Answer_10000_keywords/Answer_keywords.npy',allow_pickle = True)
    
    que_which_has_ly_keywords = []
    for sentance in data['Question']:
        sent = ' '.join(e for e in sentance.split() if e in Question_keywords)
        que_which_has_ly_keywords.append(sent)
    
     
    ans_which_has_ly_keywords = []
    for sentance in data['Answer']:
        sent = ' '.join(e for e in sentance.split() if e in Answer_keywords)
        ans_which_has_ly_keywords.append(sent)
        
    data['Question'] = que_which_has_ly_keywords
    data['Answer'] = ans_which_has_ly_keywords
        
        
    #Setting numerical value to each word in questions
    ecoded_ques = []
    for train_ques in data['Question']:
        ecoded_ques.append(list(map(lambda x: list(Question_keywords).index(x)+1,train_ques.split(" "))))
    
    ecoded_ans = []
    for train_ans in data['Answer']:
        ecoded_ans.append(list(map(lambda x: list(Answer_keywords).index(x)+1,train_ans.split(" "))))
    
    data['Question_vector'] = ecoded_ques
    data['Answer_vector'] = ecoded_ans
    
    
    #truncate and/or pad input sequences
    Max_Question_length = 24
    ques_vect_1 = sequence.pad_sequences(list(data.Question_vector), maxlen=Max_Question_length)
    ques_vect = [np.hstack((x,10001)) for x in ques_vect_1]
        
    # truncate and/or pad input sequences
    Max_Answer_length = 163
    ans_vect = sequence.pad_sequences(data.Answer_vector, maxlen=Max_Answer_length)
    
    
    #load config & weight of best model
    with open('Training_Embedding_Layer/Best_model/config.p', 'rb') as fp:
        config = pickle.load(fp)

    with open('Training_Embedding_Layer/Best_model/Best_weights_initial.p', 'rb') as fp:
        weights = pickle.load(fp)

    #loading weights to model
    import tensorflow as tf
    model = tf.keras.Model.from_config(config)
    model.set_weights(weights)
    
    ques_vect_ds = list(tf.data.Dataset.from_tensor_slices((ques_vect)).batch(1024).as_numpy_iterator())
    ans_vect_ds = list(tf.data.Dataset.from_tensor_slices((ans_vect)).batch(1024).as_numpy_iterator())
    target_ds = list(tf.data.Dataset.from_tensor_slices((data.Target)).batch(1024).as_numpy_iterator())

    loss = tf.keras.metrics.BinaryCrossentropy(name='loss')

    @tf.function
    def test_step(input_vec, labels):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(input_vec, training=False)
        loss(labels, predictions)

    def calculate_loss(ques_vect_ds,ans_vect_ds,target_ds):
        for batch in range(len(ques_vect_ds)):
            test_ques_vec = ques_vect_ds[batch]
            test_ans_vec = ans_vect_ds[batch]
            target = target_ds[batch]
            test_step([test_ques_vec,test_ans_vec], target)


    calculate_loss(ques_vect_ds,ans_vect_ds,target_ds)

    return loss.result().numpy()

In [8]:
loss = final_fun_2(train_df.iloc[:90,:])
print("Loss :" , loss)

Loss : 0.32764858
